#Team - 7
#Garvika Singh Rajawat (202318061)
#Aditi Singh (202318005)
#Jyoti Kumari (202318049)

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1b7cebca64372893544e7478ed2a80eb47ecf775e273738f92f628c886ae3540
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import col, lower, regexp_replace

In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Text Preprocessing") \
    .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df=spark.read.json('/content/drive/MyDrive/BDP mini project/English_train.json')
# df1.printSchema()
# df1.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
columns=['rumor','label']
df = df.select(*columns)
df.show()


+--------------------+---------------+
|               rumor|          label|
+--------------------+---------------+
|“#Urgent / Ramall...|        REFUTES|
|Macron to Sky New...|        REFUTES|
|Saudi Arabia evac...|        REFUTES|
|The Corona epidem...|        REFUTES|
|The official spok...|        REFUTES|
|The Prime Ministe...|        REFUTES|
|Urgent ❗️ Resigna...|        REFUTES|
|Ministry of Works...|        REFUTES|
|#Exclusively: The...|        REFUTES|
|Unconfirmed infor...|        REFUTES|
|God is Great, and...|        REFUTES|
|New innovation.. ...|        REFUTES|
|Tunisia offers to...|       SUPPORTS|
|The Spanish Casas...|       SUPPORTS|
|A citizen assault...|       SUPPORTS|
|There is no power...|       SUPPORTS|
|#Saudi Arabia bea...|       SUPPORTS|
|The death of form...|       SUPPORTS|
|Turkish demonstra...|NOT ENOUGH INFO|
|#Qais_Saeed “From...|NOT ENOUGH INFO|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Assuming spark is your SparkSession and df is your DataFrame

# Remove duplicates
df = df.dropDuplicates()

# Remove rows with NaNs in any column
df = df.dropna()
df.show()


+--------------------+---------------+
|               rumor|          label|
+--------------------+---------------+
|#Qais_Saeed “From...|NOT ENOUGH INFO|
|God is Great, and...|        REFUTES|
|#Erdogan: We will...|NOT ENOUGH INFO|
|🛑 Urgent | Jorda...|        REFUTES|
|“CAF keeps Tunisi...|NOT ENOUGH INFO|
|New innovation.. ...|        REFUTES|
|Kuwait discovers ...|       SUPPORTS|
|Urgent: Algerian ...|NOT ENOUGH INFO|
|There is no power...|       SUPPORTS|
|CAF notifies Al-A...|        REFUTES|
|Pardoning terrori...|NOT ENOUGH INFO|
|“Watch: A Qatari ...|        REFUTES|
|#Sputnik_Agency T...|        REFUTES|
|Schubert admits t...|        REFUTES|
|#Exclusively: The...|        REFUTES|
|🚨🚨|| Urgent and...|       SUPPORTS|
|Lebanon announces...|NOT ENOUGH INFO|
|News about the de...|        REFUTES|
|Isn't this also t...|NOT ENOUGH INFO|
|Tunisia offers to...|       SUPPORTS|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
# Collect the third row into a list
df.collect()[18]['rumor']


"Isn't this also the billionaire Naguib Sawiris, who has a quarter of Egypt's money and lives in its prosperity? Nor do I see anything good. Imagine with Abiy Ahmed, the Ethiopian Prime Minister, who is currently clinically dead and who usurped our right to the waters of the Nile. Ok, what's wrong with him there? https://t.co/4sY4fo3rQH"

In [ ]:
# Get the number of rows
num_rows = df.count()

# Get the number of columns
num_cols = len(df.columns)

print("Shape of the DataFrame: ({}, {})".format(num_rows, num_cols))

#No duplicates or nan values were found!

Shape of the DataFrame: (73, 2)


In [ ]:
# Step 1: Lowercase conversion
df = df.withColumn("rumor_lower", lower(df["rumor"]))

# Step 2: Regex (remove non-alphanumeric characters and special symbols)
df = df.withColumn("rumor_clean", regexp_replace(df["rumor_lower"], "[^a-zA-Z0-9\s]", ""))

In [ ]:
columns=['rumor_clean','label']
df = df.select(*columns)
df.show()

+--------------------+---------------+
|         rumor_clean|          label|
+--------------------+---------------+
|qaissaeed from no...|NOT ENOUGH INFO|
|god is great and ...|        REFUTES|
|erdogan we will t...|NOT ENOUGH INFO|
| urgent  jordan s...|        REFUTES|
|caf keeps tunisia...|NOT ENOUGH INFO|
|new innovation hi...|        REFUTES|
|kuwait discovers ...|       SUPPORTS|
|urgent algerian r...|NOT ENOUGH INFO|
|there is no power...|       SUPPORTS|
|caf notifies alah...|        REFUTES|
|pardoning terrori...|NOT ENOUGH INFO|
|watch a qatari su...|        REFUTES|
|sputnikagency the...|        REFUTES|
|schubert admits t...|        REFUTES|
|exclusively the q...|        REFUTES|
| urgent and offic...|       SUPPORTS|
|lebanon announces...|NOT ENOUGH INFO|
|news about the de...|        REFUTES|
|isnt this also th...|NOT ENOUGH INFO|
|tunisia offers to...|       SUPPORTS|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from sentence_transformers import SentenceTransformer
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import ArrayType


In [ ]:

# Load Sentence Transformer model (BERT uncased)
model_name = 'bert-base-uncased'
model = SentenceTransformer(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from pyspark.sql.functions import when

# Convert labels to numeric values
label_mapping = {"REFUTES": 0, "SUPPORTS": 1, "NOT ENOUGH INFO": 2}
df = df.withColumn("label_numeric", when(df["label"] == "REFUTES", 0)
                                   .when(df["label"] == "SUPPORTS", 1)
                                   .when(df["label"] == "NOT ENOUGH INFO", 2)
                                   .otherwise(None))

# Show the DataFrame with numeric labels
df.show()


+--------------------+---------------+-------------+
|         rumor_clean|          label|label_numeric|
+--------------------+---------------+-------------+
|qaissaeed from no...|NOT ENOUGH INFO|            2|
|god is great and ...|        REFUTES|            0|
|erdogan we will t...|NOT ENOUGH INFO|            2|
| urgent  jordan s...|        REFUTES|            0|
|caf keeps tunisia...|NOT ENOUGH INFO|            2|
|new innovation hi...|        REFUTES|            0|
|kuwait discovers ...|       SUPPORTS|            1|
|urgent algerian r...|NOT ENOUGH INFO|            2|
|there is no power...|       SUPPORTS|            1|
|caf notifies alah...|        REFUTES|            0|
|pardoning terrori...|NOT ENOUGH INFO|            2|
|watch a qatari su...|        REFUTES|            0|
|sputnikagency the...|        REFUTES|            0|
|schubert admits t...|        REFUTES|            0|
|exclusively the q...|        REFUTES|            0|
| urgent and offic...|       SUPPORTS|        

In [ ]:
from pyspark.sql.functions import col
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
import torch



In [ ]:
df.show()

+--------------------+---------------+-------------+
|         rumor_clean|          label|label_numeric|
+--------------------+---------------+-------------+
|qaissaeed from no...|NOT ENOUGH INFO|            2|
|god is great and ...|        REFUTES|            0|
|erdogan we will t...|NOT ENOUGH INFO|            2|
| urgent  jordan s...|        REFUTES|            0|
|caf keeps tunisia...|NOT ENOUGH INFO|            2|
|new innovation hi...|        REFUTES|            0|
|kuwait discovers ...|       SUPPORTS|            1|
|urgent algerian r...|NOT ENOUGH INFO|            2|
|there is no power...|       SUPPORTS|            1|
|caf notifies alah...|        REFUTES|            0|
|pardoning terrori...|NOT ENOUGH INFO|            2|
|watch a qatari su...|        REFUTES|            0|
|sputnikagency the...|        REFUTES|            0|
|schubert admits t...|        REFUTES|            0|
|exclusively the q...|        REFUTES|            0|
| urgent and offic...|       SUPPORTS|        

In [ ]:

# Define training examples from DataFrame
train_examples = df.rdd.map(lambda row: InputExample(texts=row['rumor_clean'], label=row['label_numeric'])).collect()

# Define batch size for training
train_batch_size = 2

# Create DataLoader for training data within the Spark context
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)

# Define loss function
train_loss = losses.BatchHardTripletLoss(model=model)

# Fit the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10,
)


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

Iteration:   0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
from sentence_transformers import SentenceTransformer


# Define a function to generate embeddings for a single text
def encode_text(text):
    return model.encode([text])[0].tolist()

# Define a Spark UDF to apply the encode_text function
encode_text_udf = udf(encode_text)

# Apply the UDF to generate embeddings for the 'rumor_clean' column
df_with_embeddings = df.withColumn("rumor_embeddings", encode_text_udf("rumor_clean"))

# Select only the necessary columns
df_with_embeddings = df_with_embeddings.select("rumor_clean", "rumor_embeddings", "label_numeric")

# Show the resulting DataFrame with embeddings
df_with_embeddings.show()


+--------------------+--------------------+-------------+
|         rumor_clean|    rumor_embeddings|label_numeric|
+--------------------+--------------------+-------------+
|qaissaeed from no...|[0.02240698598325...|            2|
|god is great and ...|[0.04057424142956...|            0|
|erdogan we will t...|[0.00883458741009...|            2|
| urgent  jordan s...|[-0.3275480270385...|            0|
|caf keeps tunisia...|[0.00986719131469...|            2|
|new innovation hi...|[-0.1213933303952...|            0|
|kuwait discovers ...|[-0.0267839990556...|            1|
|urgent algerian r...|[-0.2669868767261...|            2|
|there is no power...|[-0.1654892265796...|            1|
|caf notifies alah...|[-0.0339507758617...|            0|
|pardoning terrori...|[-0.0778000950813...|            2|
|watch a qatari su...|[0.08882229775190...|            0|
|sputnikagency the...|[-0.2509197294712...|            0|
|schubert admits t...|[-0.2292177975177...|            0|
|exclusively t

In [ ]:
df_with_embeddings.collect()[18][1]

'[-0.1493477076292038, 0.2219361960887909, 0.2746250331401825, -0.08009833842515945, 0.16445213556289673, -0.18389193713665009, 0.008219972252845764, 0.6313475370407104, -0.14282697439193726, -0.27079033851623535, 0.13696247339248657, -0.40590736269950867, -0.332518070936203, 0.521715521812439, 0.09867937117815018, 0.5985715389251709, 0.04785042256116867, 0.2389519065618515, -0.4129011929035187, 0.3937963545322418, 0.29213348031044006, -0.09787476807832718, 0.09454606473445892, 0.40840357542037964, 0.34607791900634766, -0.017327118664979935, -0.023029716685414314, -0.04193815588951111, -0.37294381856918335, -0.03752841055393219, 0.46796706318855286, 0.032360322773456573, -0.08399815857410431, -0.18564830720424652, -0.009075508452951908, -0.06106048822402954, -0.28264230489730835, -0.10198094695806503, -0.02330385334789753, 0.466417521238327, -0.6507440805435181, -0.4661305248737335, -0.0867471918463707, 0.08013033866882324, -0.36989980936050415, -0.49716800451278687, 0.3602680563926697

## Classification

In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.feature import VectorAssembler

# Define a UDF to convert string to vector
string_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())

# Apply the UDF to convert string to vector
df_with_embeddings = df_with_embeddings.withColumn("rumor_embeddings_vec", string_to_vector_udf("rumor_embeddings"))

# Assembling features
assembler = VectorAssembler(inputCols=["rumor_embeddings_vec"], outputCol="features")
df_assembled = assembler.transform(df_with_embeddings)

# Splitting data into training and testing sets
train_data, test_data = df_assembled.randomSplit([0.8, 0.2], seed=1234)


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, NaiveBayes, GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Logistic Regression
lr = LogisticRegression(labelCol="label_numeric", featuresCol="features", maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)

# Evaluating models
evaluator = MulticlassClassificationEvaluator(labelCol="label_numeric", predictionCol="prediction", metricName="accuracy")

# Accuracy of Logistic Regression model
lr_accuracy = evaluator.evaluate(lr_predictions)
print("Logistic Regression Accuracy:", lr_accuracy)

# F1 Score of Logistic Regression model
lr_f1 = evaluator.evaluate(lr_predictions, {evaluator.metricName: "f1"})
print("Logistic Regression F1 Score:", lr_f1)

Logistic Regression Accuracy: 0.4444444444444444
Logistic Regression F1 Score: 0.2735042735042735


In [ ]:
df_assembled.select("label_numeric").distinct().show()

+-------------+
|label_numeric|
+-------------+
|            1|
|            2|
|            0|
+-------------+



In [ ]:
from pyspark.ml.feature import StringIndexer

# StringIndexer to convert label_numeric to categorical indices
indexer = StringIndexer(inputCol="label_numeric", outputCol="indexed_label").fit(df_assembled)
df_indexed = indexer.transform(df_assembled)

# Convert indexed labels to binary format
df_indexed = df_indexed.withColumn("binary_label", when(col("indexed_label") == 2, 1).otherwise(col("indexed_label")))

# Splitting data into training and testing sets
train_data, test_data = df_indexed.randomSplit([0.8, 0.2], seed=1234)

# Gradient Boosting
gbt = GBTClassifier(labelCol="binary_label", featuresCol="features", maxIter=10)
gbt_model = gbt.fit(train_data)
gbt_predictions = gbt_model.transform(test_data)


In [ ]:
# Accuracy of Gradient Boosting model
gbt_accuracy = evaluator.evaluate(gbt_predictions)
print("Gradient Boosting Accuracy:", gbt_accuracy)

# F1 Score of Gradient Boosting model
gbt_f1 = evaluator.evaluate(gbt_predictions, {evaluator.metricName: "f1"})
print("Gradient Boosting F1 Score:", gbt_f1)

Gradient Boosting Accuracy: 0.5
Gradient Boosting F1 Score: 0.4749163879598662


In [ ]:
from pyspark.ml.feature import MinMaxScaler

# Scale features to range [0, 1]
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(train_data)
train_data_scaled = scaler_model.transform(train_data)
test_data_scaled = scaler_model.transform(test_data)

# Naive Bayes
nb = NaiveBayes(labelCol="label_numeric", featuresCol="scaled_features", smoothing=1.0, modelType="multinomial")
nb_model = nb.fit(train_data_scaled)
nb_predictions = nb_model.transform(test_data_scaled)

In [ ]:
# Evaluation
evaluator = MulticlassClassificationEvaluator(labelCol="label_numeric", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)

evaluator = MulticlassClassificationEvaluator(labelCol="label_numeric", predictionCol="prediction", metricName="f1")
nb_f1 = evaluator.evaluate(nb_predictions)
print("Naive Bayes F1 Score:", nb_f1)


Py4JJavaError: An error occurred while calling o1131.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 317.0 failed 1 times, most recent failure: Lost task 0.0 in stage 317.0 (TID 202) (9cf942be4467 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`ProbabilisticClassificationModel$$Lambda$4421/0x000000084191c840`: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage5.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext

In [ ]:
from pyspark.ml.feature import Normalizer, VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType  # Import BooleanType
import numpy as np

# Normalize the embeddings
normalizer = Normalizer(inputCol="rumor_embeddings_vec", outputCol="rumor_embeddings_normalized", p=float('inf'))
df_normalized = normalizer.transform(df_with_embeddings)

# Assembling features
assembler = VectorAssembler(inputCols=["rumor_embeddings_normalized"], outputCol="features")
df_assembled = assembler.transform(df_normalized)

# Define a UDF to check if any element in the array is NaN
contains_nan_udf = udf(lambda arr: any(np.isnan(arr)), BooleanType())  # Use BooleanType here

# Apply the UDF to check if any element in the array is NaN
df_assembled = df_assembled.withColumn("contains_nan", contains_nan_udf(col("features")))

# Filter out rows where any element in the array is NaN
df_assembled = df_assembled.filter(~col("contains_nan").cast("boolean"))  # Convert contains_nan to boolean

# Splitting data into training and testing sets
train_data, test_data = df_assembled.randomSplit([0.8, 0.2], seed=1234)

# Naive Bayes
nb = NaiveBayes(labelCol="label_numeric", featuresCol="features", smoothing=1.0, modelType="multinomial")
nb_model = nb.fit(train_data)
nb_predictions = nb_model.transform(test_data)

# Evaluation
evaluator = MulticlassClassificationEvaluator(labelCol="label_numeric", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)

evaluator = MulticlassClassificationEvaluator(labelCol="label_numeric", predictionCol="prediction", metricName="f1")
nb_f1 = evaluator.evaluate(nb_predictions)
print("Naive Bayes F1 Score:", nb_f1)


Py4JJavaError: An error occurred while calling o870.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 291.0 failed 1 times, most recent failure: Lost task 0.0 in stage 291.0 (TID 187) (9cf942be4467 executor driver): java.lang.RuntimeException: Vector values MUST NOT be Negative, NaN or Infinity, but got [0.03398686744534366,0.0470372854244608,0.03953380219052234,-0.02147372229331215,0.11153653289547803,-0.03770914180172675,-0.005307435358897726,0.1403366997778623,0.010691022149230418,-0.10873388558395192,-0.03914453732736596,-0.08107762454818858,-0.041844756336614046,0.15391470000843607,-0.04070263039163926,0.15576897691581748,0.02111579787863784,0.07082091332641692,-0.06514224347072373,0.13210665845309846,0.031593366162530855,-0.08439462985239995,0.0472091186965884,0.1451131050255109,0.0259473968688418,-0.03655801933082146,-0.02457021942597642,0.004016847145957189,-0.04629720829031469,-0.0705286139227504,0.10199105843086498,0.013194777517816779,

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("NaiveBayes") \
    .getOrCreate()

# Load your data into a DataFrame
data = spark.read.json("/content/drive/MyDrive/BDP mini project/English_train.json")
print (data)

DataFrame[_corrupt_record: string, evidence: array<array<string>>, id: string, label: string, rumor: string, timeline: array<array<string>>]


In [ ]:
# Assuming your features are in columns named 'feature1', 'feature2', ..., 'featureN'
feature_columns = ['feature1', 'feature2', ..., 'featureN']
label_column = 'label'

# VectorAssembler to combine feature columns into a single feature vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_assembled = assembler.transform(data)

# StandardScaler to scale feature vectors
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_assembled)
data_scaled = scaler_model.transform(data_assembled)

# Split the data into train and test sets
train_data, test_data = data_scaled.randomSplit([0.8, 0.2], seed=42)

# Train the Naive Bayes model
nb = NaiveBayes(labelCol=label_column, featuresCol="scaled_features", smoothing=1.0, modelType="multinomial")
nb_model = nb.fit(train_data)

# Make predictions on the test data
nb_predictions = nb_model.transform(test_data)

# Evaluate the Naive Bayes model
evaluator = MulticlassClassificationEvaluator(labelCol=label_column, predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)

# Stop the SparkSession
spark.stop()



TypeError: Invalid param value given for param "inputCols". Could not convert ['feature1', 'feature2', Ellipsis, 'featureN'] to list of strings